<a href="https://colab.research.google.com/github/LordRelentless/NGFTSimulations/blob/main/Simulation_4_1_Dark_Matter_as_an_Informational_Drag_Effect_(Animated_NGFT_v_Newtonian).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# Simulation 4.1: Dark Matter as an NGFT Informational Halo
# Language: Julia (V11 - Definitive Version with Phase Space Plot)

using Plots
using Printf
using LinearAlgebra
gr()

println("--- NGFT Galactic Rotation Curve Animation (V11 - Final) ---")

# --- 1. Define Real-World Constants & System Parameters ---
G = 6.67430e-11
SOLAR_MASS = 1.989e30
LIGHT_YEAR = 9.461e15

M_BARYONIC_GALAXY = 6e10 * SOLAR_MASS
SUN_ORBITAL_RADIUS = 27000 * LIGHT_YEAR
SUN_OBSERVED_VELOCITY = 220_000
VIRTUAL_MASS_COUPLING = 4.88

# --- 2. Simulation Fidelity Parameters ---
N_FRAMES = 1600 # Even longer to show multiple orbits
DT = 5e12       # Very high fidelity timestep for a clean, stable ellipse

# --- 3. Initialization ---
GALACTIC_CENTER = [0.0, 0.0]
sun_pos = [GALACTIC_CENTER[1] + SUN_ORBITAL_RADIUS, GALACTIC_CENTER[2]]
sun_vel = [0.0, SUN_OBSERVED_VELOCITY]

pos_history = zeros(N_FRAMES, 2)
vel_history = zeros(N_FRAMES, 2)

function calculate_force(position)
    r_vec = position - GALACTIC_CENTER
    r_mag = max(norm(r_vec), 1e12)
    m_total_effective = M_BARYONIC_GALAXY * (1 + VIRTUAL_MASS_COUPLING)
    force_vec = - (G * m_total_effective / r_mag^3) * r_vec
    return force_vec
end

# --- 4. PHASE 1: Run Physics Simulation ---
println("Phase 1: Calculating orbital physics...")
accel_current = calculate_force(sun_pos)
for i in 1:N_FRAMES
    pos_history[i, :] = sun_pos
    vel_history[i, :] = sun_vel
    sun_pos += sun_vel * DT + 0.5 * accel_current * (DT^2)
    accel_new = calculate_force(sun_pos)
    sun_vel += 0.5 * (accel_current + accel_new) * DT
    accel_current = accel_new
end
println("Phase 1 complete.")

# --- 5. PHASE 2: Post-Process Data for Plotting ---
println("Phase 2: Post-processing data for plotting...")
radius_history_ly = [norm(pos_history[i, :] - GALACTIC_CENTER) / LIGHT_YEAR for i in 1:N_FRAMES]
velocity_history_kms = [norm(vel_history[i, :]) / 1000 for i in 1:N_FRAMES]

# Calculate dynamic plot limits for the right pane
min_radius = minimum(radius_history_ly)
max_radius = maximum(radius_history_ly)
min_velocity = minimum(velocity_history_kms)
max_velocity = maximum(velocity_history_kms)

# Add some padding to the limits
radius_padding = (max_radius - min_radius) * 0.1
velocity_padding = (max_velocity - min_velocity) * 0.1

plot_xlims = (min_radius - radius_padding, max_radius + radius_padding)
plot_ylims = (min_velocity - velocity_padding, max_velocity + velocity_padding)

# Ensure the observed velocity line and expected curve are within limits
plot_ylims = (min(plot_ylims[1], SUN_OBSERVED_VELOCITY / 1000 * 0.9),
              max(plot_ylims[2], SUN_OBSERVED_VELOCITY / 1000 * 1.1))

# Ensure the expected curve is within limits (consider the full range of r_plot_ly_expected)
r_plot_ly_expected_arr = collect(r_plot_ly_expected) # Convert range to array
v_newton_curve_kms_arr = collect(v_newton_curve_kms) # Convert range to array
min_expected_velocity = minimum(v_newton_curve_kms_arr)
max_expected_velocity = maximum(v_newton_curve_kms_arr)
plot_ylims = (min(plot_ylims[1], min_expected_velocity * 0.9),
              max(plot_ylims[2], max_expected_velocity * 1.1))
plot_xlims = (min(plot_xlims[1], minimum(r_plot_ly_expected_arr) * 0.9),
              max(plot_xlims[2], maximum(r_plot_ly_expected_arr) * 1.1))


# --- 6. PHASE 3: Render the Animation ---
println("Phase 3: Rendering animation...")

r_plot_ly_expected = range(5000, 100_000, length=200)
v_newton_curve_kms = sqrt.((G * M_BARYONIC_GALAXY) ./ (r_plot_ly_expected .* LIGHT_YEAR)) ./ 1000

anim = @animate for i in 1:N_FRAMES
    p_layout = @layout [a{0.5w} b]
    p = plot(layout=p_layout, size=(1800, 800), background_color=:black,
             bottom_margin=10Plots.mm, left_margin=10Plots.mm)

    # Left Pane: Galaxy Visualization
    plot_radius_ly = 35000
    plot!(p[1], xlims=(-plot_radius_ly, plot_radius_ly), ylims=(-plot_radius_ly, plot_radius_ly),
          aspect_ratio=:equal, title="Sun's Orbit in NGFT Potential (Frame $i/$N_FRAMES)",
          legend=false, xlabel="", grid=false)
    scatter!(p[1], [0], [0], markersize=10, color=:yellow, markerstrokewidth=0)
    path_xs_ly = pos_history[1:i, 1] ./ LIGHT_YEAR
    path_ys_ly = pos_history[1:i, 2] ./ LIGHT_YEAR
    plot!(p[1], path_xs_ly, path_ys_ly, color=:cyan, alpha=0.7, lw=2)
    scatter!(p[1], [path_xs_ly[end]], [path_ys_ly[end]], markersize=6, color=:white)

    # Right Pane: The Velocity-Radius Phase Space Plot
    plot!(p[2], xlims=plot_xlims, ylims=plot_ylims, # Use dynamic limits
          title="Milky Way Rotation Curve", xlabel="Distance from Center (Light-Years)",
          ylabel="Orbital Velocity (km/s)")
    plot!(p[2], r_plot_ly_expected, v_newton_curve_kms, color=:red, lw=3, ls=:dash, label="Expected Velocity (from Visible Mass)")
    hline!(p[2], [SUN_OBSERVED_VELOCITY / 1000], color=:white, lw=2, ls=:dot, label="Observed Velocity")

    # Plot the traced path in phase space
    plot!(p[2], radius_history_ly[1:i], velocity_history_kms[1:i], color=:cyan, lw=2, label="Sun's Velocity (in NGFT Simulation)")
end

# --- 7. Save the Final Outputs ---
gif(anim, "ngft_milkyway_animated.gif", fps=30)
println("Animation saved as ngft_milkyway_animated.gif")

# Generate and save the final plot separately
println("Generating final plot frame...")
p_layout = @layout [a{0.5w} b]
final_p = plot(layout=p_layout, size=(1800, 800), background_color=:black,
         bottom_margin=10Plots.mm, left_margin=10Plots.mm)

# Left Pane: Galaxy Visualization (Final Frame)
plot_radius_ly = 35000
plot!(final_p[1], xlims=(-plot_radius_ly, plot_radius_ly), ylims=(-plot_radius_ly, plot_radius_ly),
      aspect_ratio=:equal, title="Sun's Orbit in NGFT Potential (Final Frame)",
      legend=false, xlabel="", grid=false)
scatter!(final_p[1], [0], [0], markersize=10, color=:yellow, markerstrokewidth=0)
path_xs_ly = pos_history[:, 1] ./ LIGHT_YEAR
path_ys_ly = pos_history[:, 2] ./ LIGHT_YEAR
plot!(final_p[1], path_xs_ly, path_ys_ly, color=:cyan, alpha=0.7, lw=2)
scatter!(final_p[1], [path_xs_ly[end]], [path_ys_ly[end]], markersize=6, color=:white)

# Right Pane: The Velocity-Radius Phase Space Plot (Final Frame)
plot!(final_p[2], xlims=plot_xlims, ylims=plot_ylims, # Use dynamic limits
      title="Milky Way Rotation Curve", xlabel="Distance from Center (Light-Years)",
      ylabel="Orbital Velocity (km/s)")
plot!(final_p[2], r_plot_ly_expected, v_newton_curve_kms, color=:red, lw=3, ls=:dash, label="Expected Velocity (from Visible Mass)")
hline!(final_p[2], [SUN_OBSERVED_VELOCITY / 1000], color=:white, lw=2, ls=:dot, label="Observed Velocity")

# Plot the traced path in phase space (Final Frame)
plot!(final_p[2], radius_history_ly, velocity_history_kms, color=:cyan, lw=2, label="Sun's Velocity (in NGFT Simulation)")

savefig(final_p, "ngft_rotation_curve_final.png")
println("Final plot frame saved as ngft_rotation_curve_final.png")
println("Simulation successfully finished.")

--- NGFT Galactic Rotation Curve Animation (V11 - Final) ---
Phase 1: Calculating orbital physics...
Phase 1 complete.
Phase 2: Post-processing data for plotting...
Phase 3: Rendering animation...
Animation saved as ngft_milkyway_animated.gif
Generating final plot frame...


[ Info: Saved animation to /content/ngft_milkyway_animated.gif


Final plot frame saved as ngft_rotation_curve_final.png
Simulation successfully finished.
